### RDD — практика

In [1]:
import pyspark

sc = pyspark.SparkContext("local[*]")

**Задание 1.** Ваш коллега так любит животных, что собрал у себя в загородном доме самых разных питомцев. Он пригласил вас и предложил решить задачу, какие виды животных у него обитают. Используйте метод RDD API, который покажет уникальные значения в этом RDD.

*Подсказка:* используйте метод `distinct()`.

In [2]:
pets = sc.parallelize(['кот', 'кот', 'собака', 'рыбка', 'рыбка', 'кот', 'кот', 'кот', 
                       'собака', 'кот', 'собака', 'рыбка', 'собака', 'рыбка', 'кот', 
                       'кот', 'рыбка', 'собака', 'рыбка', 'игуана','кот', 'кот', 'кот', 
                       'собака', 'рыбка', 'собака', 'рыбка', 'кот', 'кот', 'рыбка', 'собака'])

pets_dist = pets.distinct()
print(pets_dist.collect())

['кот', 'собака', 'рыбка', 'игуана']


**Задание 2.** Видов четыре, а животных гораздо больше. Поэтому коллега попросил вас найти всех котов и отфильтровать их в отдельный список, используя методы RDD API.

*Подсказка:* нужно применить метод `filter()` для значения `'кот'`, а затем получить список и распечатать его.

In [4]:
pets = pets.filter(lambda x: x == 'кот')
print(pets.collect())

['кот', 'кот', 'кот', 'кот', 'кот', 'кот', 'кот', 'кот', 'кот', 'кот', 'кот', 'кот', 'кот']


**Задание 3.** Ваш друг вошёл во вкус и теперь просит посчитать, сколько у него рыбок.

*Подсказка:* примените метод `filter()` для значения `'рыбка'`, а затем метод `count()`, чтобы распечатать результат.

In [5]:
pets = sc.parallelize(['кот', 'кот', 'собака', 'рыбка', 'рыбка', 'кот', 'кот', 'кот', 
                       'собака', 'кот', 'собака', 'рыбка', 'собака', 'рыбка', 'кот', 
                       'кот', 'рыбка', 'собака', 'рыбка', 'игуана','кот', 'кот', 'кот', 
                       'собака', 'рыбка', 'собака', 'рыбка', 'кот', 'кот', 'рыбка', 'собака'])

pets = pets.filter(lambda x: x == 'рыбка')
print(pets.count())

9


**Задание 4.** В текстовом отрывке найдите топ-10 самых часто используемых слов. Нужно применить к тексту методы RDD API, которые разделят его на отдельные слова, посчитают количество упоминаний того или иного слова и отсортируют результаты в порядке убывания.

*Подсказка:* используйте метод `flatMap()`, чтобы разбить текст на слова, метод `map()` — чтобы назначить каждому слову 1, метод `reduceByKey()` — чтобы сложить упоминания тех или иных слов вместе. В конце нужно использовать метод `sortBy()`, чтобы отсортировать результат.

In [17]:
data = sc.textFile("/datasets/wiki_extract.txt")

word_counts = data.flatMap(lambda x: x.split(' ')).map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
word_counts_sorted = word_counts.sortBy(lambda x: x[1], ascending=False)

print(word_counts_sorted.take(10))

[('the', 26), ('a', 18), ('Spark', 15), ('of', 13), ('Apache', 8), ('is', 8), ('distributed', 8), ('for', 7), ('in', 7), ('data', 6)]


**Задание 5.** В топ-10 слов из прошлой задачи, помимо существительных, есть предлоги, артикли и другие части речи, которые не отображают суть отрывка. Ваша задача — почистить результат прошлой задачи, отфильтровав часто используемые слова.

*Подсказка:* Используйте метод `map()`, чтобы применить функцию `clean_str_and_lower` к объекту data. Метод `flatMap()` поможет разбить текст на слова, `map()` — назначить каждому слову 1, `reduceByKey()` — сложить упоминания тех или иных слов вместе. А в конце обратитесь к методу `sortBy()`.

In [24]:
import pyspark
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


def clean_str_and_lower(x):
    punc = '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~-'
    lowercase_str = x.lower()
    for ch in punc:
        lowercase_str = lowercase_str.replace(ch, '')
    return lowercase_str


data = sc.textFile("/datasets/wiki_extract.txt")
stop_words = set(stopwords.words("english"))

word_counts = (
    data
    .map(clean_str_and_lower)
    .flatMap(lambda x: x.split(' '))
    .map(lambda x: (x, 1))
    .reduceByKey(lambda x, y: x + y)
)
word_counts_sorted = word_counts.sortBy(lambda x: x[1], ascending=False)
word_counts_filtered = word_counts_sorted.filter(lambda x: x[0] not in stop_words)

print(word_counts_filtered.take(10))

[('spark', 15), ('distributed', 9), ('apache', 8), ('data', 8), ('cluster', 6), ('rdd', 5), ('api', 5), ('dataset', 4), ('', 4), ('rdds', 4)]


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
